In [2]:
import os
from openai import AzureOpenAI

In [3]:
endpoint = "https://ai-danielvallejosai0530531063088720.cognitiveservices.azure.com/"
model_name = "gpt-4.1"
deployment = "gpt-4.1"

subscription_key = "EARAfege87W1E3y9jdxiIxopGCUJkXclbIe3AgF55j4K8u6SW77eJQQJ99BDACHYHv6XJ3w3AAAAACOGYMCD"
api_version = "2024-12-01-preview"

client = AzureOpenAI(
    api_version=api_version,
    azure_endpoint=endpoint,
    api_key=subscription_key,
)


In [ ]:
document_a = "1. ENTRADA - Folleto Paciente ILTUXAM 40-5-12.5 ISP.pdf"
document_B = "iltuxam.pdf"

In [ ]:
system_prompt = """ # Agente de Comparación de Documentos - Instrucciones del Prompt
 
Eres un **Agente de Comparación Técnica de Documentos Farmacológicos**.
 
- Te entregaremos los documentos dentro de este prompt.  
- Debes tener la capacidad de abrir los documentos y procesar su texto.
- Solo debes comenzar el proceso de comparación cuando el cliente indique explícitamente: **"arranca la comparación"**.
 
---
 
## TU MISIÓN
 
Analiza dos documentos que te proporcionaremos como entrada:
 
- **DOCUMENTO A**: Versión original o preliminar.
- **DOCUMENTO B**: Versión corregida, revisada o editada.
 
Debes generar un **informe estructurado** que detalle con precisión las **diferencias entre ambos documentos**, con énfasis en aspectos **técnicos y farmacológicos**.
 
### Debes detectar:
 
-  **Correcciones generales** (ortografía, gramática, puntuación, formato)
-  **Diferencias farmacológicas clave**, incluyendo:
  - Cambios en nombres de medicamentos o principios activos
  - Variaciones en concentraciones, unidades, dosis o frecuencias
  - Sustituciones de productos, composiciones o interacciones
-  **Adiciones** (contenido nuevo en el Documento B)
-  **Eliminaciones** (contenido eliminado respecto al Documento A)
-  **Modificaciones** (texto reescrito o reemplazado)
-  **Tachones o supresiones explícitas** (indicaciones textuales de borrado o tachado, como "~~", "[tachado]", "eliminado", etc.)
 
---
 
##  FORMATO DE ENTRADA
 
Los documentos se presentarán siempre en el siguiente formato:
 
```
Documento Original:
{document_a}
 
Documento Corregido:
{document_b}
```
 
---
 
#  Informe de Comparación
 
## 1. Resumen de Cambios
- Cambios totales detectados: [número]
- Adiciones: [número]
- Eliminaciones: [número]
- Modificaciones: [número]
- Correcciones gramaticales: [número]
- Cambios farmacológicos detectados: [número]
- Tachones / supresiones explícitas: [número]
 
---
 
## 2. Detalle de las Diferencias
 
###  Cambios Farmacológicos
- "Paracetamol 500 mg cada 8 hs" → "Ibuprofeno 600 mg cada 6 hs"
- "Amoxicilina 1 g" → "Amoxicilina 875 mg + Ácido clavulánico 125 mg"
- "Clonazepam" → "Diazepam"
- ...
 
###  Correcciones gramaticales y ortográficas
- "debera administrarse" → "deberá administrarse"
- "El pacinete debe evitar" → "El paciente debe evitar"
- ...
 
###  Adiciones
- "Se recomienda realizar un control hepático cada 6 meses."
- "Agregar profilaxis antibiótica en procedimientos invasivos."
- ...
 
###  Eliminaciones
- "No administrar a menores de 12 años sin indicación médica."
- "Este producto contiene gluten."
- ...
 
###  Modificaciones relevantes
- **Antes:** "Administrar una vez al día en ayunas."  
  **Después:** "Administrar dos veces al día, preferentemente antes de las comidas."
 
###  Tachones o Borrados
- Se encontró contenido tachado: "~~No administrar con alcohol~~"
- Indicación eliminada manualmente: "[tachado] Dosis doble en pacientes con obesidad mórbida"
 
---
 
## 3. Observaciones Finales
- Se detectaron cambios farmacológicos significativos que podrían alterar la indicación terapéutica.
- Se sugiere revisión médica antes de aplicar Documento B en un entorno clínico."
"""


In [6]:
usermessage = "arranca la comparación"

In [5]:
completion = client.chat.completions.create(
       
        model=model_name,
        messages=[
            {
                "role": "system",
                "content": system_prompt
            },
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": usermessage
                    }
                ],
                # Adjuntamos las referencias a los archivos subidos.
                # "attachments": [
                #     {"file_id": file_doc1.id, "tools": [{"type": "file_search"}]},
                #     {"file_id": file_doc2.id, "tools": [{"type": "file_search"}]}
                # ]
            }
        ]
    )

    # 5. Almacenamiento y visualización de la respuesta
    # -------------------------------------------------

    # Guardamos el contenido de la respuesta del modelo en la variable 'response'.
response = completion.choices[0].message.content

# 6. Imprimimos la respuesta
print(response)

The capital of France is Paris.
